# Gaussian Iterative Stockholder Analysis (GISA) method

In [ ]:
import logging

from scipy.optimize import least_squares
from setup import *

from horton_part import GaussianISAWPart

## Quadprog Solver

The problem of this solver is that the initials values are set to zeros for all pro-atom parameters and no threshold setting is availiable.

### Using API

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def quadprog_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "quadprog"
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


quadprog_solver()

### Customized Implementation

In [ ]:
import quadprog

from horton_part import check_pro_atom_parameters


def _prepare_quantities(bs_funcs, rho, propars, weights, alphas):
    nprim = len(propars)
    G = (
        2
        / np.pi**1.5
        * (alphas[:, None] * alphas[None, :]) ** 1.5
        / (alphas[:, None] + alphas[None, :]) ** 1.5
    )
    G = (G + G.T) / 2

    # a = np.zeros(nprim, float)
    # for k in range(nprim):
    #     a[k] = 2 * np.einsum("i,i,i", weights, bs_funcs[k], rho)
    a = 2 * np.einsum("i,ni,i->n", weights, bs_funcs, rho)

    # Construct linear equality or inequality constraints
    C = np.zeros([nprim, nprim + 1])
    # First column : corresponds to the EQUALITY constraint sum_{k=1..Ka} c_(a,k)= N_a
    C[:, 0] = np.ones(nprim)
    # Other K_a columns : correspond to the INEQUALITY constraints c_(a,k) >=0
    C[0:nprim, 1 : (nprim + 1)] = np.identity(nprim)
    b = np.zeros(nprim + 1)
    # First coefficient : corresponds to the EQUALITY constraint sum_{k=1..Ka} c_(a,k) = N_a
    pop = np.einsum("i,i", weights, rho)
    b[0] = pop
    return G, a, C, b


def opt_propars_qp_quadprog(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize parameters for proatom density functions using quadratic programming.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr


        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = 2 * \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.


    """
    G, a, C, b = _prepare_quantities(bs_funcs, rho, propars, weights, alphas)
    result = quadprog.solve_qp(G=G, a=a, C=C, b=b, meq=1)[0]
    check_pro_atom_parameters(result, total_population=float(b[0]))
    return result

Now we can call the customized solver through follows:

In [ ]:
def customized_quadprog_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_quadprog
    kwargs["inner_threshold"] = 1e-8
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_quadprog_solver()

## CVXOPT Solver
Another sovler implemented in `CVXOPT` can also be applied.

### Using API

In [ ]:
def cvxopt_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "cvxopt"
    kwargs["solver_options"] = {"eps_rel": 1e-8}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


cvxopt_solver()

### Customized Implementation

In [ ]:
import cvxopt


def opt_propars_qp_cvxopt(bs_funcs, rho, propars, points, weights, alphas, threshold):
    """
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    nprim, npt = bs_funcs.shape

    S = (
        2
        / np.pi**1.5
        * (alphas[:, None] * alphas[None, :]) ** 1.5
        / (alphas[:, None] + alphas[None, :]) ** 1.5
    )
    P = cvxopt.matrix(S)

    vec_b = np.zeros((nprim, 1), float)
    for k in range(nprim):
        vec_b[k] = 2 * np.einsum("i,i", weights * bs_funcs[k], rho)
    q = -cvxopt.matrix(vec_b)

    # Linear inequality constraints
    G = cvxopt.matrix(0.0, (nprim, nprim))
    G[:: nprim + 1] = -1.0
    h = cvxopt.matrix(0.0, (nprim, 1))

    # Linear equality constraints
    A = cvxopt.matrix(1.0, (1, nprim))
    pop = np.einsum("i,i", weights, rho)
    b = cvxopt.matrix(pop, (1, 1))

    # initial_values = cvxopt.matrix(np.array([1.0] * nprim).reshape((nprim, 1)))
    opt_CVX = cvxopt.solvers.qp(
        P,
        q,
        G,
        h,
        A,
        b,
        # initvals=propars,
        initvals=np.zeros_like(propars),
        options={"feastol": threshold, "show_progress": 0},
        # options={"show_progress": log.do_medium, "feastol": threshold},
    )
    new_propars = np.asarray(opt_CVX["x"]).flatten()
    check_pro_atom_parameters(new_propars, total_population=float(pop))
    return new_propars

In [ ]:
def customized_cvxopt_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_cvxopt
    kwargs["inner_threshold"] = 1e-8
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_cvxopt_solver()

## ECOS Sovler

In [ ]:
def ecos_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "ecos"
    kwargs["solver_options"] = {"reltol": 1e-10}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


ecos_solver()

## OSQP Solver

In [ ]:
def osqp_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "osqp"
    kwargs["solver_options"] = {"eps_rel": 1e-14}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


osqp_solver()

More supported sovlers can be found in [`qpsovlers`](https://qpsolvers.github.io/qpsolvers/supported-solvers.html) package.

## Customized SLSQP Solver

A equivalent method is using `minimization` with the 'SLSQP' sovler implemented in `SciPy` package. 

In [ ]:
def opt_propars_qp_slsqp(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize parameters for proatom density functions using quadratic programming.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr


        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = 2 * \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.


    """
    G, a, C, b = _prepare_quantities(bs_funcs, rho, propars, weights, alphas)
    meq = 1

    def _obj_func(x):
        return 0.5 * np.dot(x, G).dot(x) - np.dot(a, x)

    constraints = []
    if C is not None:
        constraints = [
            {
                "type": "eq" if i < meq else "ineq",
                "fun": lambda x, C=C, b=b, i=i: (np.dot(C.T, x) - b)[i],
            }
            for i in range(C.shape[1])
        ]

    result = minimize(
        _obj_func,
        x0=np.zeros(len(G)),
        method="SLSQP",
        constraints=constraints,
        tol=threshold,
        options={"maxiter": 2000},
    )
    return result.x

In [ ]:
def customized_slsqp_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_cvxopt
    # the value is from the test file of quadprog package.
    kwargs["inner_threshold"] = 1e-10
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_slsqp_solver()

## Customized Least-Square Sovler

Users can easily apply customized solvers. The first step involves defining a customized solver, for example, `customized_solver`. The corresponding arguments and their meanings are as follows:

In [ ]:
def opt_propars_lstsq(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr

        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """

    def _obj_func(x):
        pro = np.sum(x[:, None] * bs_funcs, axis=0)
        return np.sqrt(weights) * np.abs(pro - rho)

    result = least_squares(
        _obj_func, x0=np.zeros_like(propars), bounds=(0, np.inf), ftol=threshold
    )
    return result.x

Next, we need to pass this customized solver to our partitioning method.

In [ ]:
def customized_lstsq_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_lstsq
    kwargs["inner_threshold"] = 1e-14
    part = GaussianISAWPart(**kwargs)
    part.do_all()

    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_lstsq_solver()

The result of the least squares method differs slightly from the quadratic programming method due to the constraints. We can observe that the deviation in the total electrons obtained by the least squares method is larger than that obtained by the quadratic programming method.